### 1. Dataset

Los datos de origen son proporcionados en un archivos csv:

* udfs: dataset con datos de operaciones financieras.


In [1]:
import pandas as pd

udfs = pd.read_csv("udfs.csv",sep=";")
udfs

,nb,contract,udf_ref,fmly,grp,type,country,udf_name,num_value,string_value,date_value,data_timestamp_part,data_date_part,source_system
0,444444,3333,28786653,IRD,LN_BR,NaN,ESP,M_CCY,NaN,NaN,NaN,20201128041303,2020-12-30,Mx3EU
1,2222222,2222222,2222222,IRD,IRS,NaN,ESP,M_CRDTCHRG,30.0,NaN,NaN,20210203032054,2020-12-30,Mx3EU
2,2222222,2222222,2222222,IRD,IRS,NaN,ESP,M_SELLER,NaN,LB_TLECLER,NaN,20210203032054,2020-12-30,Mx3EU
3,2222222,2222222,2222222,IRD,IRS,NaN,ESP,M_LIQDTYCHRG,50.0,NaN,NaN,20210203032054,2020-12-30,Mx3EU
4,2222222,2222222,2222222,IRD,IRS,NaN,ESP,M_MVA,20.0,NaN,NaN,20210203032054,2020-12-30,Mx3EU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,10000009,10000009,10000009,IRD,BOND,FWD,ESP,M_SUCURSAL,NaN,1999,NaN,20210203032054,2020-12-30,Mx3EU
382,10000009,10000009,10000009,IRD,BOND,FWD,ESP,M_RVA,0.0,NaN,NaN,20210203032054,2020-12-30,Mx3EU
383,10000009,10000009,10000009,IRD,BOND,FWD,ESP,M_DIRECTIAV,0.0,NaN,NaN,20210203032054,2020-12-30,Mx3EU
384,10000009,10000009,10000009,IRD,BOND,FWD,ESP,M_CLIENT,NaN,NaN,NaN,20210203032054,2020-12-30,Mx3EU


### 2. Columnas y significado:

* nb: número de referencia de la operación.
* contract: identificador de contrato.
* udf_ref: identificador de operación de trading.
* fmly: familia a la que pertenece la operación financiera.
* grp: grupo al que pertenece la operación financiera.
* type: tipo de operación financiera.
* country: país de origen de la operación.
* udf_name: campo informado en el registro.
* num_value: valor numérico.
* string_value: valor de cadena de caracteres.
* date_value: valor de fecha.
* data_timestamp_part: marca temporal.
* data_date_part: fecha en la que se almacena la información.
* source_system: fuente de los datos.


### 3. Descripción del problema:

Si hacemos una visión general a nuestro conjunto de datos, podemos observar como hay hasta 10 registros (filas) para cada valor de *nb*, donde cada registro solo da información para un valor de *udf_name*. Esto es un gasto innecesario de almacenamiento y computación, además de complicar los futuros cálculos derivados de estos datos. Por esta razón, necesitamos convertir estos registros con el mismo *nb* a un solo registro.

Nuestro dataframe final tendrá que contener las siguientes columnas: `nb, M_CCY, M_CLIENT, M_CRDTCHRG, M_DIRECTIAV, M_DISCMARGIN, M_LIQDTYCHRG, M_MVA, M_RVA, M_SELLER, M_SUCURSAL`

* nb: debe contener el número de referencia de la operación.
* M_CLIENT, M_SELLER, M_CCY, M_SUCURSAL: deben mapear el valor de *string_value*
* M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, , M_MVA, M_RVA: deben mapear el valor de *num_value*


Una vez tengamos este resultado, necesitaremos eliminar las operaciones que no tengan informados ninguno de los siguientes campos:

M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, M_MVA, M_RVA, M_SELLER

No informados en este caso significa que o son valores nulos, vacíos o 0, en el caso de los campos numéricos.

### 4. Reto:

* Obtener un dataframe final que contenga las columnas indicadas, con un registro por *nb* y con los valores correctos mapeados.
* Las operaciones con los campos M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, , M_MVA, M_RVA, M_SELLER no informados no deben existir.
* Hacerlo de la manera más eficiente posible a nivel computacional.

**NOTA:** Cada uno de los pasos descritos en el problema pueden efectuarse en una sola línea.